In [2]:
!pip install langchain langchain_openai faiss-gpu --upgrade --quiet

<a href = "https://colab.research.google.com/github/qxr777/llm-application-code/blob/master/retrieval_augmented_generation/rag-helloworld.ipynb"
target = "_parent"> Open In Colab </a>

In [ ]:
# 三选一 加载OPENAI_API_KEY

# 本地运行，包含.env文件
# from dotenv import load_dotenv
# load_dotenv()

# 本地运行，不包含.env文件
# import os
# os.environ['OPENAI_API_KEY'] = 'sk-123456789'

# google colab运行，需要配置secret key
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

In [4]:
# 创建LLM
model = 'gpt-3.5-turbo-0613'
llm = ChatOpenAI(model=model)

In [5]:
# 创建Prompt
prompt = ChatPromptTemplate.from_template('基于上下文：{context}\n回答：{input}')

In [6]:
# 创建输出解析器
output_parser = StrOutputParser()

In [11]:
# 模拟文档
docs = [Document(page_content="课程代码 01041110	课程名称 企业级应用软件设计与开发	课程大类 专业学位硕士类课程	班级 23硕士合班	学生数 165	周次 2-13	时间 【星期二】6,7,8节	上课地点 马区西-东教学楼-233(西东教-合四)	")]

In [12]:
# 文档嵌入
splits = RecursiveCharacterTextSplitter().split_documents(docs)
vector_store = FAISS.from_documents(splits, OpenAIEmbeddings())
retriever = vector_store.as_retriever()

In [13]:
# 创建Chain
chain_no_context = RunnablePassthrough() | llm | output_parser
chain = (
    {"context": retriever, "input": RunnablePassthrough()}
    | prompt | llm | output_parser
)

In [14]:
# 调用Chain
question = '企业级应用软件设计与开发的上课地点在哪里？'
print(chain_no_context.invoke(question))
print(chain.invoke(question))

上课地点一般在大学或专业培训机构的教室或实验室进行。具体的上课地点可以根据培训机构或大学的安排而定，可以通过咨询相关机构或学校获取具体信息。
企业级应用软件设计与开发的上课地点在马区西-东教学楼-233(西东教-合四)。
